# Deep Learning in Medicine
## BMSC-GA 4493, BMIN-GA 3007
## Homework 3: RNNs

Note 1: If you don't know how to run jupyter on the Prince cluster, here is another step-by-step guide here: 
<a href='https://docs.google.com/document/d/1HIdtzqJ6-RpsV0z2Gf5iXphNBTRca1kHZPlyqFxKpWs/edit?usp=sharing'> **Running Jupyter on the Cluster **</a>

Note 2: If you need to write mathematical terms, you can type your answeres in a Markdown Cell via LaTex
See: <a href="https://stackoverflow.com/questions/13208286/how-to-write-latex-in-ipython-notebook">here</a> if you have issues. To see basic LaTex notation see: <a href="https://en.wikibooks.org/wiki/LaTeX/Mathematics">here</a>.


Submission instruction: Upload and Submit your final jupyter notebook file in newclasses.nyu.edu

**Submission deadline: Thursday April 16th 2020 5pm.**



# Question 1: Literature Review: DeepMod (Total points 20 + 10 bonus points)

Read this paper:

#### Qian Liu, Li Fang, Guoliang Yu, Depeng Wang, Chuan-Le Xiao & Kai Wang, _"Detection of DNA base modifications by deep recurrent neural network on Oxford Nanopore sequencing data"_ ,  Nature Communications, 2019 https://www.nature.com/articles/s41467-019-10168-2

We are interested in understanding the task, the methods that is proposed in this publication, technical aspects of the implementation, and possible future work.

**1.1) (5 points)** After you read the full article, go back to section **Methods**. Briefly describe the primary Deep Learning architecture used in DeepMod. Write the number of layers used, number of features input to the network?


**1.2) (5 points)** Describe the optional second deep neural network architecuture, including the number of layers and number of input features, number of nodes in hidden layers. 

**1.3) (5 points)** What is the loss function used to train the primary network? 

What are the evaluation criteria used by the authors for all the tasks? (**Hint:** Look at Performance measurements)

**1.4) (5 points)** Are there some data augmentation/regularization that authors have used? What are some techniques that could have been used and wasn't?

**1.5) (Bonus maximum 10 points)**. What other architectures would you try? For each family of models, please do a literature search and see if a paper on that architecture for the task of DNA base modification has been used.

# Question 2: Literature Review: Self Attention (20 points + 10 bonus points)

Read this paper: 


#### Xianlong Zeng, Yunyi Feng, Soheil Moosavinasab, Deborah Lin, Simon Lin, Chang Liu _"Multilevel Self-Attention Model and its Use on Medical Risk Prediction"_ https://psb.stanford.edu/psb-online/proceedings/psb20/Zeng.pdf

After you read the paper, go back to Section 3.2 and 3.3. 

**2.1) (10 points)** Describe the architecture used in the paper to generate patient embedding. Please mention the architecture of self-attention units including any formula given in paper. Also include the input to the architecture.  


**2.2) (5 points)** What are the different tasks that the architecture is used to solve? What are the Loss functions used for different tasks? What are the evaluation criteria for the different tasks?

**2.3) (5 points)** In Section 5.2 What is the best model according to the evaluation criteria? How is it different from the second best model?

**2.4) (Bonus maximum 10 points)** What are some alternative architectures/Loss functions/Pretraining methods that you would recommend as followup work? Name 2 potential architectures, and in a few sentences explain why the proposed changes might work better.


# Question 3 - Programming: Build Classifiers on Medical Transcriptions - Recurrent Neural Networks and Self Attention(60 points + 10 bonus points)

Let's build some models now. In this homework, we will focus on a dataset which has around 5000 medical transcriptions and the corresponding medical specialty. The data is available <a href="https://www.kaggle.com/tboyle10/medicaltranscriptions">here</a>.

Here, we will focus on predicting top few classes of medical specialty, from the transcription text. <a href="https://github.com/nyumc-dl/BMSC-GA-4493-Spring2020/tree/master/lab5">Lab 5</a> will be very useful here.

**3.1) (5 points)** Read the csv using Pandas. Select the top 5 classes ('medical_specialty') from the data. Only keep the rows that belong to one of these classes in your data. Which classes are there, and how many rows do you have after this filteration?

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('mtsamples.csv',index_col=0)
df.head()

,description,medical_specialty,sample_name,transcription,keywords
0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller..."
1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh..."
2,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 1,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ...","bariatrics, laparoscopic gastric bypass, heart..."
3,2-D M-Mode. Doppler.,Cardiovascular / Pulmonary,2-D Echocardiogram - 1,"2-D M-MODE: , ,1. Left atrial enlargement wit...","cardiovascular / pulmonary, 2-d m-mode, dopple..."
4,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 2,1. The left ventricular cavity size and wall ...,"cardiovascular / pulmonary, 2-d, doppler, echo..."


In [2]:
df.groupby('medical_specialty').count().sort_values('sample_name',ascending=False).head()

,description,sample_name,transcription,keywords
medical_specialty,,,,
Surgery,1103,1103,1088,1036
Consult - History and Phy.,516,516,516,234
Cardiovascular / Pulmonary,372,372,371,281
Orthopedic,355,355,355,303
Radiology,273,273,273,251


In [3]:
top_5_classes = df.groupby('medical_specialty').count().sort_values('sample_name',ascending=False)[:5].index.tolist()
top_5_classes

[' Surgery',
 ' Consult - History and Phy.',
 ' Cardiovascular / Pulmonary',
 ' Orthopedic',
 ' Radiology']

In [4]:
df = df.loc[df['medical_specialty'].apply(lambda x : x in top_5_classes),:]
df = df.loc[df['transcription'].apply(lambda x : type(x) == str),:]
df['label'] = df['medical_specialty'].apply(lambda x : top_5_classes.index(x))
df.head()

,description,medical_specialty,sample_name,transcription,keywords,label
3,2-D M-Mode. Doppler.,Cardiovascular / Pulmonary,2-D Echocardiogram - 1,"2-D M-MODE: , ,1. Left atrial enlargement wit...","cardiovascular / pulmonary, 2-d m-mode, dopple...",2
4,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 2,1. The left ventricular cavity size and wall ...,"cardiovascular / pulmonary, 2-d, doppler, echo...",2
7,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 3,"2-D ECHOCARDIOGRAM,Multiple views of the heart...","cardiovascular / pulmonary, 2-d echocardiogram...",2
9,Echocardiogram and Doppler,Cardiovascular / Pulmonary,2-D Echocardiogram - 4,"DESCRIPTION:,1. Normal cardiac chambers size....","cardiovascular / pulmonary, ejection fraction,...",2
11,"Normal left ventricle, moderate biatrial enla...",Cardiovascular / Pulmonary,2-D Doppler,"2-D STUDY,1. Mild aortic stenosis, widely calc...","cardiovascular / pulmonary, 2-d study, doppler...",2


**3.2) (5 points)** Now convert your data into train, test and validation set. Shuffle the rows, and split them with ratios of (train:60%, valid:20%, test:20%). Set the random seed to 2020. Please follow the steps from https://pytorch.org/docs/stable/notes/randomness.html to set all the seeds to make the results reproducible.

In [5]:
from sklearn.model_selection import train_test_split
import torch
def seed_torch(seed=2020):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

In [6]:
train_data, test_val_data = train_test_split(df, test_size=0.40, random_state=2020)
test_data, val_data = train_test_split(test_val_data, test_size=0.50, random_state=2020)
train_data.index = np.arange(len(train_data))
val_data.index = np.arange(len(val_data))
test_data.index = np.arange(len(test_data))

**3.3) (5 points)** Create a function to create vocabulary from the training data. Only use the transcription column for this. Use the tokenization scheme of your choice and create a vocabulary.

In [7]:
from collections import Counter
import re
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

def build_vocab(sentences, min_count=3, max_vocab=None):
    UNK = "<UNK>"
    PAD = "<PAD>"
    word_count = Counter()
    for sentence in sentences:
        sentence = re.sub('[\\(\[#.!?,\'\/\])0-9]', ' ', sentence)
        word_count.update(word_tokenize(sentence.lower()))
    
    vocabulary = list([w for w in word_count if word_count[w] > min_count]) + [UNK, PAD]
    indices = dict(zip(vocabulary, range(len(vocabulary))))
    return vocabulary, indices

vocabulary, vocab_indices = build_vocab(train_data['transcription'])
print(len(vocabulary))

[nltk_data] Downloading package punkt to /home/yz6432/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


8714


**3.4) (10 points)** Write a dataloader and collate function so that we can begin to train our networks! You can choose to use either the complete transcription text or fix a maximum length of transcription text as input for your model.

In [8]:
from torch.utils.data import DataLoader, Dataset

class mtDataset(Dataset):
    def __init__(self, vocab_index, df, label = 'label'):
        self.vocab_index = vocab_index
        self.df = df
        self.label = label
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, key):
        sentence = self.df.loc[key, 'transcription']
        sentence = re.sub('[\\(\[#.!?,\'\/\])0-9]', ' ', sentence)
        token_indices = np.array([self.vocab_index[word] if word in self.vocab_index else self.vocab_index['<UNK>'] for word in word_tokenize(sentence.lower())])
        return (torch.tensor(token_indices) , self.df.loc[key, self.label])


def pad_collate(batch):
    (xx, yy) = zip(*batch)
    x_lens = [len(x) for x in xx]

    xx_pad = pad_sequence(xx, batch_first=True, padding_value=len(vocabulary)-1)

    return torch.as_tensor(xx_pad), torch.as_tensor(x_lens), torch.LongTensor(yy)
    

BATCH_SIZE = 32
train_loader = DataLoader(mtDataset(vocab_indices, train_data),
                          batch_size=BATCH_SIZE,
                          shuffle=True,
                          collate_fn = pad_collate)
val_loader = DataLoader(mtDataset(vocab_indices, val_data),
                         batch_size=BATCH_SIZE,
                         shuffle=True,
                         collate_fn = pad_collate)

**3.5) (10 points)** Now you are ready to build your sequence classification model!

First, Build a simple GRU model that takes as input the text indices from the vocabulary, and ends with a softmax over total number of classes. Use the embedding and hidden dimension of your choice. 

**Please train your model to reach at the least 55% accuracy on the test set.**

At each epoch, compute and print **Average Cross Entropy loss** and **Accuracy** on both **train and validation set** 

Plot your validation and train loss over different epochs. 

Plot your validation and train accuracies over different epochs. 

Finally print accuracy on the test set.

In [9]:
import time
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score
from torch.nn.utils.rnn import pad_sequence
from torch.nn.utils.rnn import pack_padded_sequence
device = torch.device("cpu")

class RNN(nn.Module):
    def __init__(self, hidden_dim=40, output_dim=5, vocab_size=len(vocabulary), embedding_dim=50, rnn='GRU'):
        super(RNN, self).__init__()
        
        self.emb = nn.Embedding(vocab_size, embedding_dim, padding_idx=vocab_size-1)
        self.hidden_dim = hidden_dim
        self.rnn_fn = rnn
        assert self.rnn_fn in ['LSTM','GRU']
        self.rnn = getattr(nn, rnn)(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x, x_len):
        x = self.emb(x)
        _, last_hidden = self.rnn(pack_padded_sequence(x, x_len, batch_first=True, enforce_sorted=False))
        if self.rnn_fn == 'LSTM':
            last_hidden = last_hidden[0]
        out = self.fc(last_hidden.view(-1, self.hidden_dim))
        return out

In [10]:
def train(model, train_loader=train_loader, val_loader=val_loader, learning_rate=0.001, num_epoch=10):
    start_time = time.time()
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=10**(-5))
    train_loss_return = []
    train_acc_return = []
    val_loss_return = []
    val_acc_return = []
    
    for epoch in range(num_epoch):
        # Training steps
        correct = 0
        total = 0
        predictions = []
        truths = []
        model.train()
        train_loss_list = []
        for i, (data, data_len, labels) in enumerate(train_loader):
            data, data_len, labels = data.to(device), data_len.to(device), labels.to(device)
            outputs = model(data, data_len)
            pred = outputs.data.max(-1)[1]
            predictions += list(pred.cpu().numpy())
            truths += list(labels.cpu().numpy())
            total += labels.size(0)
            correct += (pred == labels).sum()
            model.zero_grad()
            loss = loss_fn(outputs.squeeze(), labels)
            train_loss_list.append(loss.item())
            loss.backward()
            optimizer.step()
        # report performance
        acc = (100 * correct / total)
        train_acc_return.append(acc)
        train_loss_every_epoch = np.average(train_loss_list)
        train_loss_return.append(train_loss_every_epoch)
        print('----------Epoch{:2d}/{:2d}----------'.format(epoch,num_epoch))
        print('Train set | Loss: {:6.4f} | Accuracy: {:4.2f}% '.format(train_loss_every_epoch, acc))
        
        # Evaluate after every epochh
        correct = 0
        total = 0
        model.eval()
        predictions = []
        truths = []
        val_loss_list = []
        with torch.no_grad():
            for i, (data, data_len, labels) in enumerate(val_loader):
                data, data_len, labels = data.to(device), data_len.to(device), labels.to(device)
                outputs = model(data, data_len)
                loss = loss_fn(outputs.squeeze(), labels)
                val_loss_list.append(loss.item())
                pred = outputs.data.max(-1)[1]
                predictions += list(pred.cpu().numpy())
                truths += list(labels.cpu().numpy())
                total += labels.size(0)
                correct += (pred == labels).sum()
            # report performance
            acc = (100 * correct / total)
            val_acc_return.append(acc)
            val_loss_every_epoch = np.average(val_loss_list)
            val_loss_return.append(val_loss_every_epoch)
            elapse = time.strftime('%H:%M:%S', time.gmtime(int((time.time() - start_time))))
            print('Test set | Loss: {:6.4f} | AUC: {:4.2f}% | time elapse: {:>9}'.format(val_loss_every_epoch, acc,elapse))
    return model,train_loss_return,train_acc_return,val_loss_return,val_acc_return

In [11]:
gru_model = RNN().to(device)
model,train_loss_return,train_acc_return,val_loss_return,val_acc_return = train(gru_model)

KeyboardInterrupt: 

**3.6) (25 points)** Now, let's finetune a sequence classification model based on BERT. Please install the Huggingface's Transformers library for this. Use the Pretrained 'bert-base-uncased' model for this problem. Please use the BERT tokenizer from the pretrained built for 'bert-base-uncased' model . Use the AdamW optimizer from the transformers library for optimization. Remember BERT uses Attention masks for input so you need to create a separate dataloader for BERT. Please keep in mind that BERT can handle maximum of 512 tokens.

**Please finetune the model so that it reaches at least 65% accuracy on the test set.**

The rest of your experimental setting should be the same as 3.5:

At each epoch, compute and print **Average Cross Entropy loss** and **Accuracy** on both **train and validation set** 

Plot your validation and train loss over different epochs. 

Plot your validation and train accuracies over different epochs. 

Finally print accuracy on the test set.

**3.7) (Bonus maximum 10 points)** List 5 examples on the test set that BERT misclassified. Describe reasons identified for misclassification.